In [9]:
from keras.layers import merge, Input, Dense, Convolution2D, MaxPooling3D, Flatten, Lambda
from keras.models import Model
from keras import backend as K

In [68]:
hand_conv_1 = Convolution2D(16, 5, 5, activation='relu', dim_ordering='th', border_mode='same')
hand_pool_1 = MaxPooling3D(pool_size=(2,2,2), dim_ordering='th')
hand_conv_2 = Convolution2D(32, 5, 5, activation='relu', dim_ordering='th', border_mode='same')
hand_pool_2 = MaxPooling3D(pool_size=(2,2,2), dim_ordering='th')
hand_conv_3 = Convolution2D(48, 4, 4, activation='relu', dim_ordering='th', border_mode='same')
hand_pool_3 = MaxPooling3D(pool_size=(2,2,2), dim_ordering='th')

main_conv_1 = Convolution2D(16, 5, 5, activation='relu', dim_ordering='th', border_mode='same')
main_pool_1 = MaxPooling3D(pool_size=(2,2,2), dim_ordering='th')
main_conv_2 = Convolution2D(32, 5, 5, activation='relu', dim_ordering='th', border_mode='same')
main_pool_2 = MaxPooling3D(pool_size=(2,2,2), dim_ordering='th')
main_conv_3 = Convolution2D(48, 4, 4, activation='relu', dim_ordering='th', border_mode='same')
main_pool_3 = MaxPooling3D(pool_size=(2,2,2), dim_ordering='th')

pack_and_expand = Lambda(lambda x: K.expand_dims(K.pack(x, axis=1), 1))
unpack = Lambda(lambda x: K.unpack(x, axis=1), unpack_shape)
def unpack_shape(input_shape):
    return [(64, 64)] * input_shape[1]
squeeze = Lambda(lambda x: K.squeeze(x, 1))

In [65]:
K.unpack(inputs_hand[0], axis=1)

[<tf.Tensor 'unpack_659:0' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_659:1' shape=(?, 64, 64) dtype=float32>]

In [69]:
unpack(inputs_hand[0])

Exception: The `get_output_shape_for` method of layer "lambda_63"" should return one shape tuple per output tensor of the layer. Found: [((64, 64), (64, 64))]

In [11]:
inputs_hand = [Input(shape=(2, 64, 64)) for i in range(32)]
inputs_main = [Input(shape=(2, 64, 64)) for i in range(32)]

hand_conv_1_out = [K.unpack(hand_conv_1(i), axis=1) for i in inputs_hand]
hand_pool_1_in = [K.expand_dims(K.pack(c, axis=1), 1) for c in zip(*hand_conv_1_out)]
hand_conv_2_in = [K.squeeze(hand_pool_1(c), 1) for c in hand_pool_1_in]
hand_conv_2_out = [K.unpack(hand_conv_2(i), axis=1) for i in hand_conv_2_in]
hand_pool_2_in = [K.expand_dims(K.pack(c, axis=1), 1) for c in zip(*hand_conv_2_out)]
hand_conv_3_in = [K.squeeze(hand_pool_2(c), 1) for c in hand_pool_2_in]
hand_conv_3_out = [K.unpack(hand_conv_3(i), axis=1) for i in hand_conv_3_in]
hand_pool_3_in = [K.expand_dims(K.pack(c, axis=1), 1) for c in zip(*hand_conv_3_out)]
hand_pool_3_out = [K.squeeze(hand_pool_3(c), 1) for c in hand_pool_3_in]

main_conv_1_out = [K.unpack(main_conv_1(i), axis=1) for i in inputs_main]
main_pool_1_in = [K.expand_dims(K.pack(c, axis=1), 1) for c in zip(*main_conv_1_out)]
main_conv_2_in = [K.squeeze(main_pool_1(c), 1) for c in main_pool_1_in]
main_conv_2_out = [K.unpack(main_conv_2(i), axis=1) for i in main_conv_2_in]
main_pool_2_in = [K.expand_dims(K.pack(c, axis=1), 1) for c in zip(*main_conv_2_out)]
main_conv_3_in = [K.squeeze(main_pool_2(c), 1) for c in main_pool_2_in]
main_conv_3_out = [K.unpack(main_conv_3(i), axis=1) for i in main_conv_3_in]
main_pool_3_in = [K.expand_dims(K.pack(c, axis=1), 1) for c in zip(*main_conv_3_out)]
main_pool_3_out = [K.squeeze(main_pool_3(c), 1) for c in main_pool_3_in]

In [14]:
merged = K.pack(hand_pool_3_out + main_pool_3_out, axis=1)
shape = merged.get_shape()
flattened = Lambda(lambda x: K.reshape(x, [-1, 98304]))(merged)
penultimate = Dense(512, activation='relu')(flattened)
ultimate = Dense(20, activation='softmax')(penultimate)

In [37]:
hand_conv_1(inputs_hand[0])

<tf.Tensor 'Relu_646:0' shape=(?, 16, 64, 64) dtype=float32>

In [34]:
tf.unpack(hand_conv_1(inputs_hand[0]), axis=1)

[<tf.Tensor 'unpack_644:0' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:1' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:2' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:3' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:4' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:5' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:6' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:7' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:8' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:9' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:10' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:11' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:12' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:13' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:14' shape=(?, 64, 64) dtype=float32>,
 <tf.Tensor 'unpack_644:15' shape=(?, 64, 64) dtype=float32>]

Exception: The `get_output_shape_for` method of layer "lambda_15"" should return one shape tuple per output tensor of the layer. Found: [(2,)]

In [8]:

model = Model(input=inputs_hand + inputs_main, output=output)

Exception: Output tensors to a Model must be Keras tensors. Found: Tensor("Softmax_1:0", shape=(?, 20), dtype=float32)

In [100]:
Flatten()(merge(hand_pool_3_out, mode='concat'))

<tf.Tensor 'Reshape_615:0' shape=(?, ?) dtype=float32>